# 먼저 tensorflow GPU 확인!

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11011498139758127047
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3059430195
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3295971302436839120
physical_device_desc: "device: 0, name: GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [1]:
import tensorflow as tf
tf.debugging.set_log_device_placement(True)

# 텐서 생성
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [2]:
#%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
    raise SystemError('GPU device not found')

def cpu():
    with tf.device('/cpu:0'):
        random_image_cpu = tf.random.normal((200,224,224,3))
        net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
        return tf.math.reduce_sum(net_cpu)

def gpu():
    with tf.device('/device:GPU:0'):
        random_image_gpu = tf.random.normal((200,224,224,3))
        net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above. [Op:Conv2D]

In [3]:
import glob
from PIL import Image
from numpy import expand_dims
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil

# 이진화 전처리 함수

세웅이 모델에 들어가는 이미지 형태는 최종 BGR이여야 함!

In [4]:
# 1, 원본 그대로~


def original(imgpath):
    img = load_img(img_path)
    img = img_to_array(img)
    bgr_image = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
    return bgr_image

In [5]:
# adaptive threshold만 적용
def threshold(imgpath):
    # grayscale 변환
    img = load_img(imgpath)
    img = img_to_array(img)
    gray_image = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #(a,b) -> 2차원으로 변환
    gray_image = gray_image.astype('uint8')
    
    # adaptive threshold 적용
    th = cv2.adaptiveThreshold(gray_image,
                               255,
                               cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                               cv2.THRESH_BINARY,
                               99,
                               15)
    
    # bgr 변환
    bgr_image = cv2.cvtColor(th,cv2.COLOR_GRAY2BGR)
    return bgr_image

In [6]:
# morph gradient
def morphology(imgpath):
    # grayscale 변환
    img = load_img(imgpath)
    img = img_to_array(img)
    gray_image = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #(a,b) -> 2차원으로 변환
    gray_image = gray_image.astype('uint8')
    
    # morphgradient 적용
    kernel = np.ones((2,2), np.uint8)
    img_grad = cv2.morphologyEx(gray_image, cv2.MORPH_GRADIENT, kernel)
    
    # bgr로 변환
    bgr_image = cv2.cvtColor(img_grad,cv2.COLOR_GRAY2BGR)
    return bgr_image

In [7]:
# 이미지 대비 향상 -> adaptive threshold
def threshold_sole(imgpath):
    # grayscale 변환
    img = load_img(imgpath)
    img = img_to_array(img)
    gray_image = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #(a,b) -> 2차원으로 변환
    gray_image = gray_image.astype('uint8')
    
    # 이미지 대비를 향상
    image_enhanced = cv2.equalizeHist(gray_image)
    
    # Adaptive Thresholding 적용 
    max_output_value = 255   # 출력 픽셀 강도의 최대값
    neighborhood_size = 99
    subtract_from_mean = 15
    image_binarized = cv2.adaptiveThreshold(image_enhanced,
                                          255,
                                          cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                          cv2.THRESH_BINARY,
                                          99,
                                          15)
    
    # bgr 변환
    bgr_image = cv2.cvtColor(image_binarized,cv2.COLOR_GRAY2BGR)  
    return bgr_image

In [8]:
# morphology -> adaptive threshold
def morph_threshold(imgpath):
    # graysacle
    img = load_img(imgpath)
    img = img_to_array(img)
    gray_image = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray_image = gray_image.astype('uint8')
    
    # morph gradient
    kernel = np.ones((2,2), np.uint8)
    img2_grad = cv2.morphologyEx(gray_image, cv2.MORPH_GRADIENT, kernel)
    
    # adaptive threshold
    th = cv2.adaptiveThreshold(img2_grad, 
                               255, 
                               cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                               cv2.THRESH_BINARY,
                               99,
                               15)
    
    # bgr 변환
    bgr_image = cv2.cvtColor(th,cv2.COLOR_GRAY2BGR) # result함수 돌리기 위해 3차원으로 변환 
    return bgr_image

In [9]:
# morph_gradient -> 이미지 대비 향상 -> adaptive threshold
def morph_threshold_sole(imgpath):
    #grayscale
    img = load_img(imgpath)
    img = img_to_array(img)
    gray_image = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray_image = gray_image.astype('uint8')
    
    # 이미지 대비를 향상
    image_enhanced = cv2.equalizeHist(gray_image)
    
    # morph_gradient
    kernel = np.ones((2,2), np.uint8)
    img2_grad = cv2.morphologyEx(image_enhanced, cv2.MORPH_GRADIENT, kernel)
    
    # Adaptive Thresholding 적용 
    image_binarized = cv2.adaptiveThreshold(img2_grad,
                                          255,
                                          cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                          cv2.THRESH_BINARY,
                                          99,
                                          15)
    
    # bgr 변환
    bgr_image = cv2.cvtColor(image_binarized,cv2.COLOR_GRAY2BGR)

    
    return bgr_image

# 원본 이미지 경로 불러오기

In [20]:
# 카테고리 재분류한 폴더 경로
folder_path = "C:/Users/seohyeonpark/프로젝트/튜닝스타 카테고리 재분류"

# 코드에 맞게 이름 변경해서 생성할 폴더 경로
new_folder_path = "C:/Users/seohyeonpark/프로젝트/TuningStar2"

In [24]:
for i in range(2, 93):
    i_images = glob.glob(folder_path + '/새 폴더 (' + str(i) + ')/*.jpg' )
    
    j = 0
    for path in i_images:
        category_num = i-2
        img_name = str(category_num) + " (" + str(j) + ").jpg"
        to_folder_path = new_folder_path + '/' + img_name
        shutil.copy(path, to_folder_path)
        j += 1

In [25]:
categories_with_a_image = glob.glob(folder_path + '/새 폴더/*.jpg' )

category_num = 91
for path in categories_with_a_image:
    img_name = str(category_num) + ".jpg"
    to_folder_path = new_folder_path + '/' + img_name
    shutil.copy(path, to_folder_path)
    category_num += 1

In [78]:
my_img_pathes = glob.glob("C:/Users/seohyeonpark/프로젝트/TuningStar2/*.jpg")

In [79]:
# 경로 확인
my_img_pathes[:5] # '서현' 로컬 컴퓨터에서는 주소 마지막에 \\ 뜸..

['C:/Users/seohyeonpark/프로젝트/TuningStar2\\0 (0).jpg',
 'C:/Users/seohyeonpark/프로젝트/TuningStar2\\0 (1).jpg',
 'C:/Users/seohyeonpark/프로젝트/TuningStar2\\0 (2).jpg',
 'C:/Users/seohyeonpark/프로젝트/TuningStar2\\0 (3).jpg',
 'C:/Users/seohyeonpark/프로젝트/TuningStar2\\1 (0).jpg']

In [80]:
# 주소 정상적으로 불러지면 이 셀은 건너 뛰어도 됨
i = 0
for path in my_img_pathes:
    path = path.replace('\\','/')
    my_img_pathes[i] = path
    i += 1

In [29]:
my_img_pathes[:5]

['C:/Users/seohyeonpark/프로젝트/TuningStar2/0 (0).jpg',
 'C:/Users/seohyeonpark/프로젝트/TuningStar2/0 (1).jpg',
 'C:/Users/seohyeonpark/프로젝트/TuningStar2/0 (2).jpg',
 'C:/Users/seohyeonpark/프로젝트/TuningStar2/0 (3).jpg',
 'C:/Users/seohyeonpark/프로젝트/TuningStar2/1 (0).jpg']

In [81]:
len(my_img_pathes)

621

# 분류할 카테고리 넘버 리스트, 딕셔너리 만들기
#### category_nums, category_count_dict

In [84]:
category_nums=[]
for img_path in my_img_pathes:
    num_jpg = img_path.split("/")[-1] # 휠 파일 이름.jpg
    category_num = num_jpg.split('.')[0] # jpg 떼어 버리기!
    try:
        category_num = int(category_num) # (i)번 째 안되어 있으면 바로 정수형으로 바꿔잇!
    except:
        if "(" in category_num: # (i)번째 표시되어있으면 날리고 앞에 카테고리만 저장
            category_num = int(category_num.split("(")[0])
        elif "-" in category_num: # - i 로 표시한 사람것도 날리고 앞에 카테고리만 저장
            category_num = int(category_num.split('-')[0])
    category_nums.append(category_num)

In [85]:
category_nums

[0,
 0,
 0,
 0,
 1,
 1,
 1,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 13,
 14,
 14,
 14,
 14,
 14,
 14,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 17,
 17,
 17,
 17,
 17,
 17,
 18,
 18,
 19,
 19,
 19,
 19,
 2,
 2,
 20,
 20,
 21,
 21,
 21,
 21,
 22,
 22,
 23,
 23,
 23,
 24,
 24,
 24,
 25,
 25,
 25,
 26,
 26,
 27,
 27,
 27,
 28,
 28,
 28,
 29,
 29,
 29,
 3,
 3,
 30,
 30,
 31,
 31,
 31,
 31,
 32,
 32,
 32,
 33,
 33,
 33,
 34,
 34,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 36,
 36,
 36,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 

각 카테고리 번호 별로 몇 개씩 들어있는 지 카운팅한 뒤, 카테고리(key):등장횟수(value) 형태로 딕셔너리 생성

In [90]:
category_count_dict = {}
for i in range(len(category_nums)):
    num_count = category_nums.count(i)
    category_count_dict[i] = num_count

In [91]:
count_list = []
for value in category_count_dict.values():
    if value not in count_list:
        count_list.append(value)

In [92]:
def prime_number(number):
    if number != 1:
        for f in range(2, number):
            if number % f == 0:
                return False
    else:
        return False
    return True

In [93]:
print(count_list)

[4, 3, 2, 14, 22, 5, 12, 7, 6, 11, 43, 15, 10, 21, 32, 19, 8, 37, 52, 0]


In [94]:
count = []
for i in count_list:
    count.append(prime_number(i))
print(np.array(count) * np.array(count_list))
print(len(count))
print(count)# 소수 : 2, 3, 5, 7 -> 다 곱하면 210

[ 0  3  2  0  0  5  0  7  0 11 43  0  0  0  0 19  0 37  0  0]
20
[False, True, True, False, False, True, False, True, False, True, True, False, False, False, False, True, False, True, False, True]


In [65]:
2*3*5*7*11*43*19*37

69828990

Keras의 ImageDataGenerator를 통해 Data Augmentation을 할 수 있는 generator생성.

In [4]:
## data augmentation
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications import Xception
from keras.utils import multi_gpu_model

Using TensorFlow backend.


In [98]:
tf.test.gpu_device_name()

'/device:GPU:0'

# 각 전처리 방법으로 이진화 후 다른 폴더에 저장

배정받은 이미지 경로들을 통해 이미지를 가져오고,
하나의 카테고리를 하나의 폴더로 만듦. 위에서도 이야기했듯이 파일 번호 = 카테고리이므로 이미 카테고리 폴더가 생성되어있는 경우는 넘어가게 햇음.

In [123]:
# 하나의 이미지에 대해 파일 이름, 카테고리 인덱스, 같은 카테고리로 분류된 이미지의 개수 반환해주는 함수
def count_category(img_path, category_count_dict):
    # 1. 파일 이름(img_name)
    num_jpg = img_path.split("/")[-1] # 이미지 경로 마지막 (휠 번호.jpg) 만 떼어내기
    category_num = num_jpg.split('.')[0] # .jpg떼어 버리기! (휠 번호.jpg) -> (휠 번호)
    img_name = category_num
    
    # 2. 카테고리 인덱스(category_num)
    try:
        '''
        카테고리 번호만 category_num에 저장하도록 한다 -> 나중에 augmentation한 폴더 이름 설정할 때 사용
        현재 img_name은 1 or 1 (1) or 1-1 형식으로 되어있음.
        맨 앞에 카테고리 분류한 숫자에만 관심이 있기 때문에
        먼저 1과 같이 같은 카테고리 내에 한 개의 이미지만 있는 경우는 int(category_num)으로 숫자 추출,
        에러가 발생한다면 "(", ")","-" 등이 category_num에 포함되어 있는 것이기 때문에,
        각 문장부호 들로 split 후 맨 앞에 카테고리 분류 숫자를 반환
        '''
        category_num = int(category_num) # (i)번 째 안되어 있으면 바로 정수형으로 바꿔잇!
    except:
        if "(" in category_num: # (i)번째 표시되어있으면 날리고 앞에 카테고리만 저장
            category_num = int(category_num.split("(")[0])
        elif "-" in category_num: # - i 로 표시한 사람것도 날리고 앞에 카테고리만 저장
            category_num = int(category_num.split('-')[0])
    
    # 3. 같은 카테고리로 분류된 이미지의 개수(category_count)
    category_count = category_count_dict[category_num] # 해당 이미지의 카테고리에 몇개의 이미지가 있는가
    return img_name, category_count, category_num

# 이미지 전처리 함수
#: 앞서 정해준 이진화 방법들 중 하나를 function에 넣어주면 이미지를 해당 방법으로 이진화 후 
def preprocess(img_path, function):
    img = function(img_path) # function에 전처리 함수를 넣어 해당 방법으로 전처리한다.
    x = expand_dims(img,0) # 차원 추가
    return x

def augmentation(x):
    with tf.device('/device:GPU:0'):
        data = datagen.flow(x,
                            batch_size=1,
                            save_to_dir=folder_path,
                            save_prefix=category_num,
                            save_format="jpeg")
        return data

In [129]:
''' 전처리 함수 메뉴
1. original : 원본 이미지
2. threshold : Adaptive만 적용
3. morphology : morph gradient만 적용
4. threshold_sole : 이미지 대비 향상 -> Adaptive
5. morph_threshold : morph gradient -> Adaptive 
6. morph_threshold_sole : 이미지 대비 향상 morph gradient -> adaptive threshold 
'''


##ImageDataGenerator에 원하는 옵션들을 추가해주면 옵션들 적용하면서 랜덤하게 생성해줍니다.
with tf.device('/device:GPU:0'):
    datagen = ImageDataGenerator(
            rotation_range = 90,
            width_shift_range = 0.2,
            height_shift_range = 0.2,
            brightness_range = [0.05 , 2.0],
            vertical_flip = True,
            horizontal_flip = True,
            fill_mode= 'nearest')
    # 전처리 함수 자기 파트 # 빼고 진행하기
    #function = original # 선민
    #function = threshold # 세웅
    #function = threshold_sole # 예림
    #function = morph_threshold # 솔
    function = morph_threshold_sole # 서현

    for img_path in my_img_pathes:

        ''' category별 개수 구하기 '''
        img_name, category_count, category_num = count_category(img_path, category_count_dict)

        ''' 전처리 '''

        x = preprocess(img_path, function)

        ''' Augment & Save '''

        # augmentation 개수
        img_num = 1800 # 1680 : 210의 배수로 해줘야 각 폴더에 들어가는 이미지 수가 최대한 비슷해짐
        aug_num = int(img_num/category_count)
        print(category_num, '_category count : ', category_count)
        print(category_num, '_aug_num : ', aug_num)

        # if img_num%category_count != 0: # 여기서 print의 결과로 나오는 이미지들은 10000의 약수가 아니므로, augmentation한 결과가 정확히 10000장이 되지 못함. 해당 카테고리에서는 수동으로 이미지 복붙해서 10000장 맞춰주세요.
        #    print(category_num)


        # 아래 폴더 이름은 "C:/Users/seohyeonpark/프로젝트" 부분만 원하는 경로로 바꿔주세요. 
        folder_path = "C:/Users/seohyeonpark/프로젝트/Data Augmentation2" + "/" + str(category_num) # folder_path는 자기가 저장할 경로에 맞게 수정해주어야 함. str(category)는 건들지 말고 앞부분만 수정.
        if not(os.path.isdir(folder_path)):
            os.makedirs(folder_path)
        j=0
        for batch in datagen.flow(x,
                                  batch_size=1,
                                  save_to_dir=folder_path,
                                  save_prefix=category_num,
                                  save_format="jpeg"):
            j+=1
            if j > aug_num:
                break

0 _category count :  4
0 _aug_num :  450
0 _category count :  4
0 _aug_num :  450
0 _category count :  4
0 _aug_num :  450
0 _category count :  4
0 _aug_num :  450
1 _category count :  3
1 _aug_num :  600
1 _category count :  3
1 _aug_num :  600
1 _category count :  3
1 _aug_num :  600
10 _category count :  12
10 _aug_num :  150
10 _category count :  12
10 _aug_num :  150
10 _category count :  12
10 _aug_num :  150
10 _category count :  12
10 _aug_num :  150
10 _category count :  12
10 _aug_num :  150
10 _category count :  12
10 _aug_num :  150
10 _category count :  12
10 _aug_num :  150
10 _category count :  12
10 _aug_num :  150
10 _category count :  12
10 _aug_num :  150
10 _category count :  12
10 _aug_num :  150
10 _category count :  12
10 _aug_num :  150
10 _category count :  12
10 _aug_num :  150
11 _category count :  4
11 _aug_num :  450
11 _category count :  4
11 _aug_num :  450
11 _category count :  4
11 _aug_num :  450
11 _category count :  4
11 _aug_num :  450
12 _category 

39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
4 _category count :  2
4 _aug_num :  900
4 _category count :  2
4 _aug_num :  900
40 _category co

53 _category count :  4
53 _aug_num :  450
54 _category count :  3
54 _aug_num :  600
54 _category count :  3
54 _aug_num :  600
54 _category count :  3
54 _aug_num :  600
55 _category count :  2
55 _aug_num :  900
55 _category count :  2
55 _aug_num :  900
56 _category count :  3
56 _aug_num :  600
56 _category count :  3
56 _aug_num :  600
56 _category count :  3
56 _aug_num :  600
57 _category count :  4
57 _aug_num :  450
57 _category count :  4
57 _aug_num :  450
57 _category count :  4
57 _aug_num :  450
57 _category count :  4
57 _aug_num :  450
58 _category count :  3
58 _aug_num :  600
58 _category count :  3
58 _aug_num :  600
58 _category count :  3
58 _aug_num :  600
59 _category count :  11
59 _aug_num :  163
59 _category count :  11
59 _aug_num :  163
59 _category count :  11
59 _aug_num :  163
59 _category count :  11
59 _aug_num :  163
59 _category count :  11
59 _aug_num :  163
59 _category count :  11
59 _aug_num :  163
59 _category count :  11
59 _aug_num :  163
59 _

73 _category count :  6
73 _aug_num :  300
73 _category count :  6
73 _aug_num :  300
73 _category count :  6
73 _aug_num :  300
73 _category count :  6
73 _aug_num :  300
74 _category count :  2
74 _aug_num :  900
74 _category count :  2
74 _aug_num :  900
75 _category count :  2
75 _aug_num :  900
75 _category count :  2
75 _aug_num :  900
76 _category count :  3
76 _aug_num :  600
76 _category count :  3
76 _aug_num :  600
76 _category count :  3
76 _aug_num :  600
77 _category count :  2
77 _aug_num :  900
77 _category count :  2
77 _aug_num :  900
78 _category count :  2
78 _aug_num :  900
78 _category count :  2
78 _aug_num :  900
79 _category count :  3
79 _aug_num :  600
79 _category count :  3
79 _aug_num :  600
79 _category count :  3
79 _aug_num :  600
8 _category count :  4
8 _aug_num :  450
8 _category count :  4
8 _aug_num :  450
8 _category count :  4
8 _aug_num :  450
8 _category count :  4
8 _aug_num :  450
80 _category count :  2
80 _aug_num :  900
80 _category count 

### 각 카테고리 별로 augmentation 이미지 15개 씩 augmented_validation 폴더에 저장

In [130]:
category_num = []
for i in category_nums:
    if i not in category_num:
        category_num.append(i)

In [131]:
import random
for i in category_num:
    ith_images = glob.glob("C:/Users/seohyeonpark/프로젝트/Data Augmentation2" + "/" + str(i) + "/*.jpeg")
    remove_images_pathes = random.sample(ith_images, len(ith_images) - 1500)
    for path in remove_images_pathes:
        os.remove(path)
                           

In [132]:
import random
for category in category_num:
    # 해당 category 폴더에서 무작위로 선택된 15개의 이미지 경로를 저장 (경로 지정해줘야 함)
    folder_path = "C:/Users/seohyeonpark/프로젝트/Data Augmentation2" + "/" + str(category)
    
    all_images_pathes = glob.glob(folder_path+"/*.jpeg")
    
    # 아래 부분은 위의 my_image_pathes와 같이 주소가 정상적으로 안 불러져 왔을 경우에만 실행
    i = 0
    for path in all_images_pathes:
        path = path.replace('\\','/')
        all_images_pathes[i] = path
        i += 1
    
    selected_images_pathes = random.sample(all_images_pathes, 111)
    
    # 이동할 폴더 생성(경로 지정해줘야 함)
    to_images_pathes = "C:/Users/seohyeonpark/프로젝트/augmented_validation2" + "/" + str(category)
    os.makedirs(to_images_pathes)
    
    for selected_image_path in selected_images_pathes:
        file_name = selected_image_path.split('/')[-1]
        from_image_path = folder_path + "/" + file_name
        shutil.move(from_image_path, to_images_pathes)

### 원본 이미지로 validation set 만들기

In [133]:
for i in range(len(my_img_pathes)):
    from_image_path = my_img_pathes[i]
    file_name = my_img_pathes[i].split('/')[-1]
    category_num = file_name.split('.')[0]
    
    
    try:
        category_num = int(category_num) # (i)번 째 안되어 있으면 바로 정수형으로 바꿔잇!
    except:
        if "(" in category_num: # (i)번째 표시되어있으면 날리고 앞에 카테고리만 저장
            category_num = int(category_num.split("(")[0])
        elif "-" in category_num: # - i 로 표시한 사람것도 날리고 앞에 카테고리만 저장
            category_num = int(category_num.split('-')[0])
    
    to_images_pathes = "C:/Users/seohyeonpark/프로젝트/original_validation2" + "/" + str(category_num)
    if not(os.path.isdir(to_images_pathes)):
        os.makedirs(to_images_pathes)
    shutil.copy(from_image_path, to_images_pathes + "/" + file_name)

# 이후에는 세웅이 모델 학습 붙이기

In [35]:
##alphanumeric order
def sort(lst): 
    lst = [str(i) for i in lst] 
    lst.sort() 
    lst = [int(i) if i.isdigit() else i for i in lst ] 
    return lst 

## training data에 label 붙여주기
num_list = []
for i in range(91):
    num_list.append(i)

## wheel_imgs_num dictionary : {카테고리 번호 : 카테고리 속하는 이미지 갯수}
wheel_imgs_num = {}
for idx in num_list:
    wheel_img_path = glob.glob('C:/Users/seohyeonpark/프로젝트/Data Augmentation2/%s/*.jpeg' % idx) ## %s 부분은 '카테고리 번호명인 폴더'를 의미하니, 참고하여 경로를 설정해주시면 됩니다.  
    wheel_imgs_num[idx] = len(wheel_img_path)
    
## 빈 array를 만들고, alphanumeric order로 레이블 만들기 : wheel_imgs_num.keys()를 sort함수 적용하면 alphanumeric order로 순회합니다.
## 빈 array에 np.concatenate로 numpy 배열 붙여나가는 방식으로 labeling 했습니다.
label_trained = np.array([], dtype ='int32')
for idx in sort(wheel_imgs_num.keys()):
    x = np.array([idx]*wheel_imgs_num[idx], dtype ='int32') ## [카테고리 번호]를 카테고리 속하는 이미지 갯수만큼 배열 만들기
    label_trained = np.concatenate((label_trained,x)) ## 배열 붙이기
    
del wheel_imgs_num
del wheel_img_path
del x

In [6]:
pip install image_classifiers==0.2.2

Note: you may need to restart the kernel to use updated packages.


In [5]:
from classification_models.resnet import ResNet50, preprocess_input
from skimage.transform import resize
from PIL import Image
model = ResNet50(input_shape=(224,224,3), weights='imagenet', include_top=False)

In [8]:
def result(img,model):
    x = resize(img,(224,224))*255
    x = preprocess_input(x)
    x = np.expand_dims(x,0)
    y = model.predict(x)
    y = np.mean(y, axis=(1,2))
    return y

In [38]:
with tf.device('/device:GPU:0'):
    train_img_array = np.zeros((len(label_trained),2048))
    pathes = glob.glob("C:/Users/seohyeonpark/프로젝트/Data Augmentation2/*")
    # 아래 부분은 위의 my_image_pathes와 같이 주소가 정상적으로 안 불러져 왔을 경우에만 실행
    idx = 0
    for path in pathes:
        path = path.replace('\\','/')
        pathes[idx] = path
        idx += 1
    sorted_pathes = sort(pathes)
    del pathes

    for folder_path in sorted_pathes:
        img_pathes = glob.glob(folder_path+"/*.jpeg")
        #j = 0
        #for path in img_pathes:
        #    path = path.replace('\\','/')
        #    img_pathes[i] = path
        #    j += 1

        for i in range(len(img_pathes)):
            img = Image.open(img_pathes[i])
            img_numpy = np.array(img)
            img_vector = result(img_numpy,model)
            train_img_array[i] = img_vector
            del img
            del img_numpy
            del img_vector
        print(folder_path+"is done.")

    np.save('C:/Users/seohyeonpark/프로젝트/feature vector/bottleneck_features_train.npy', train_img_array)

C:/Users/seohyeonpark/프로젝트/Data Augmentation2/0is done.
C:/Users/seohyeonpark/프로젝트/Data Augmentation2/1is done.
C:/Users/seohyeonpark/프로젝트/Data Augmentation2/10is done.
C:/Users/seohyeonpark/프로젝트/Data Augmentation2/11is done.
C:/Users/seohyeonpark/프로젝트/Data Augmentation2/12is done.
C:/Users/seohyeonpark/프로젝트/Data Augmentation2/13is done.
C:/Users/seohyeonpark/프로젝트/Data Augmentation2/14is done.
C:/Users/seohyeonpark/프로젝트/Data Augmentation2/15is done.
C:/Users/seohyeonpark/프로젝트/Data Augmentation2/16is done.
C:/Users/seohyeonpark/프로젝트/Data Augmentation2/17is done.
C:/Users/seohyeonpark/프로젝트/Data Augmentation2/18is done.
C:/Users/seohyeonpark/프로젝트/Data Augmentation2/19is done.
C:/Users/seohyeonpark/프로젝트/Data Augmentation2/2is done.
C:/Users/seohyeonpark/프로젝트/Data Augmentation2/20is done.
C:/Users/seohyeonpark/프로젝트/Data Augmentation2/21is done.
C:/Users/seohyeonpark/프로젝트/Data Augmentation2/22is done.
C:/Users/seohyeonpark/프로젝트/Data Augmentation2/23is done.
C:/Users/seohyeonpark/프로젝트/Data Au

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/seohyeonpark/프로젝트/feature vector/bottleneck_features_train.npy'

In [39]:
np.save('C:/Users/seohyeonpark/프로젝트/feature_vector/bottleneck_features_train.npy', train_img_array)

In [2]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:

with tf.device('/device:GPU:0'):
    num_list = []
    for i in range(91):
        num_list.append(i)

    ## wheel_imgs_num dictionary : {카테고리 번호 : 카테고리 속하는 이미지 갯수}
    wheel_imgs_num = {}

    for idx in num_list:
        wheel_img_path = glob.glob('C:/Users/seohyeonpark/프로젝트/augmented_validation2/%s/*.jpeg' % idx) ## %s 부분은 '카테고리 번호명인 폴더'를 의미하니, 참고하여 경로를 설정해주시면 됩니다.  
        wheel_imgs_num[idx] = len(wheel_img_path)

    ## 빈 array를 만들고, alphanumeric order로 레이블 만들기 : wheel_imgs_num.keys()를 sort함수 적용하면 alphanumeric order로 순회합니다.
    ## 빈 array에 np.concatenate로 numpy 배열 붙여나가는 방식으로 labeling 했습니다.
    label_validation = np.array([], dtype ='int32')
    for idx in sort(wheel_imgs_num.keys()):
        x = np.array([idx]*wheel_imgs_num[idx], dtype ='int32') ## [카테고리 번호]를 카테고리 속하는 이미지 갯수만큼 배열 만들기
        label_validation = np.concatenate((label_validation,x)) ## 배열 붙이기

    del wheel_imgs_num
    del wheel_img_path
    del x

    def result(img,model):
        x = resize(img,(224,224))*255
        x = preprocess_input(x)
        x = np.expand_dims(x,0)
        y = model.predict(x)
        y = np.mean(y, axis=(1,2))
        return y

    validation_img_array = np.zeros((len(label_validation),2048))
    pathes = glob.glob("C:/Users/seohyeonpark/프로젝트/augmented_validation2/*")
    sorted_pathes = sort(pathes)
    del pathes
    for folder_path in sorted_pathes:
        img_pathes = glob.glob(folder_path+"/*.jpeg")
        for i in range(len(img_pathes)):
            img = Image.open(img_pathes[i])
            img_numpy = np.array(img)
            img_vector = result(img_numpy,model)
            train_img_array[i] = img_vector
            del img
            del img_numpy
            del img_vector
        print(folder_path+"is done.")

    np.save('C:/Users/seohyeonpark/프로젝트/feature vector/bottleneck_features_validation.npy', validation_img_array)